In [38]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np

In [39]:
# Create the optimization model
model = gb.Model("Question 1: Binary")

In [40]:
c = list(range(1, 25))

In [41]:
c = np.array(c)
c = c.reshape(4,-1)

In [42]:
c

array([[ 1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12],
       [13, 14, 15, 16, 17, 18],
       [19, 20, 21, 22, 23, 24]])

In [43]:
x = model.addVars(4, 6, lb=0, vtype=GRB.BINARY, name="Routes")
y = model.addVars(1, 6, lb=0, vtype=GRB.BINARY, name="Routes Opened")

In [44]:
objective = gb.quicksum(c[i,j]*x[i,j] for i in range (4) for j in range (6))
model.setObjective(objective, GRB.MINIMIZE)

In [45]:
#Only one company can be assigned to a route if it is opened
for j in range(6):
    model.addConstr(gb.quicksum(x[i, j] for i in range(4)) == y[0, j], "Assign 1 Company a Route")

In [46]:
#Each company can be assigned to at most two bus routes
for i in range(4):
    model.addConstr(gb.quicksum(x[i,j] for j in range (6)) <= 2, "Assign 2 Routes to a Company Max")

In [47]:
#At least three bus routes must be opened
model.addConstr(gb.quicksum(y[0, j] for j in range(6)) >= 3, "At Least 3 Routes assigned")

<gurobi.Constr *Awaiting Model Update*>

In [48]:
#Route 2 and 5 must both be opened or not at all
model.addConstr(y[0, 1] == y[0, 4], "Routes 2 and 5 Opened")

<gurobi.Constr *Awaiting Model Update*>

In [49]:
#Either route 3 must be opened or route 4 must be opened but not both
model.addConstr(y[0, 2] + y[0, 3] == 1, "Routes 3 or 4 Opened")

<gurobi.Constr *Awaiting Model Update*>

In [50]:
#If company B is assigned to route 1, it cannot also provide service to route 4
model.addConstr(x[1, 0] <= 1 - x[1, 3], "Company B Routes 1 or 4 Opened")

<gurobi.Constr *Awaiting Model Update*>

In [51]:
#If company A is assigned to route 3, it must also be assigned to route 5.
model.addConstr(x[0, 2] <= x[0, 4], "Company A Routes 2 and 5")

<gurobi.Constr *Awaiting Model Update*>

In [52]:
#Company D must be assigned to at least one route
model.addConstr(gb.quicksum(x[3, j] for j in range (6)) >= 1, "At least 1 company D")

<gurobi.Constr *Awaiting Model Update*>

In [53]:
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 16 rows, 30 columns and 74 nonzeros
Model fingerprint: 0xcab263d3
Variable types: 0 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 54.0000000
Presolve removed 4 rows and 6 columns
Presolve time: 0.01s
Presolved: 12 rows, 24 columns, 56 nonzeros
Found heuristic solution: objective 35.0000000
Variable types: 0 continuous, 24 integer (24 binary)
Found heuristic solution: objective 28.0000000

Root relaxation: objective 2.550000e+01, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |   

In [54]:
# Print the decision variables
print(model.printAttr('X'))


    Variable            X 
-------------------------
 Routes[0,1]            1 
 Routes[0,4]            1 
 Routes[3,2]            1 
Routes Opened[0,1]            1 
Routes Opened[0,2]            1 
Routes Opened[0,4]            1 
None
